<a href="https://colab.research.google.com/github/nilaykd/playground/blob/main/tensorflow_fun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 737.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 142.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 145.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.1 MB/s eta 0:00:00


In [8]:
import pandas as pd
import tensorflow as tf
import numpy as np
import string

In [2]:
# Load raw data
texts = []
labels = []
with open("/content/drive/MyDrive/Datasets/SMSSpamCollection", "r") as infile:
  for l in infile:
    label, text = l.strip().split("\t")
    if label == "ham":
      labels.append(0)
    else:
      labels.append(1)
    text = "".join([ch.lower() for ch in text if ch not in string.punctuation])
    texts.append(text)

labels = np.asarray(labels)

# Keras preprocessing

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from sklearn.model_selection import train_test_split

tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(texts)

X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2)

X_train_emb = tokenizer.texts_to_sequences(X_train)
X_test_emb = tokenizer.texts_to_sequences(X_test)

maxlen = 100

X_train = pad_sequences(X_train_emb, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test_emb, padding='post', maxlen=maxlen)

vocab_size = len(tokenizer.word_index) + 1

# A simple neural network

In [15]:
import keras
import tensorflow as tf

embedding_dim = 50
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size,embedding_dim))
model.add(keras.layers.GlobalMaxPool1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])
model.summary()

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath="/content/drive/MyDrive/Projects/0_playground/2_Playing with Tensorflow/checkpoints/weights.{epoch:02d}-{val_loss:.2f}.weights.h5",
                                                 save_weights_only=True,
                                                 verbose=1)
model.fit(X_train, y_train,
          epochs=100,
          verbose=False,
          validation_data=(X_test, y_test),
          batch_size=16,
          callbacks=[cp_callback])
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
model.save("/content/drive/MyDrive/Projects/0_playground/2_Playing with Tensorflow/models/TFmodel.keras")

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_4          │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


Epoch 1: saving model to /content/drive/MyDrive/Projects/0_playground/2_Playing with Tensorflow/checkpoints/weights.01-0.23.weights.h5

Epoch 2: saving model to /content/drive/MyDrive/Projects/0_playground/2_Playing with Tensorflow/checkpoints/weights.02-0.08.weights.h5

Epoch 3: saving model to /content/drive/MyDrive/Projects/0_playground/2_Playing with Tensorflow/checkpoints/weights.03-0.05.weights.h5

Epoch 4: saving model to /content/drive/MyDrive/Projects/0_playground/2_Playing with Tensorflow/checkpoints/weights.04-0.04.weights.h5

Epoch 5: saving model to /content/drive/MyDrive/Projects/0_playground/2_Playing with Tensorflow/checkpoints/weights.05-0.04.weights.h5

Epoch 6: saving model to /content/drive/MyDrive/Projects/0_playground/2_Playing with Tensorflow/checkpoints/weights.06-0.04.weights.h5

Epoch 7: saving model to /content/drive/MyDrive/Projects/0_playground/2_Playing with Tensorflow/checkpoints/weights.07-0.04.weights.h5

Epoch 8: saving model to /content/drive/MyDrive